# Obtain training data

# Train a tokenizer

In [7]:
#!python3 -m pip install tokenizers

from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./.data/train").glob("**/*.txt")] + [str(x) for x in Path("./.data/eval").glob("**/*.txt")]
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

tokenizer.save(".models/EsperBERTo-small", "")

['.models/EsperBERTo-small/-vocab.json',
 '.models/EsperBERTo-small/-merges.txt']

# Train a language model from scratch

In [13]:
from torch.utils.data.dataset import Dataset

class EsperantoDataset(Dataset):
    def __init__(self, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            "./models/EsperBERTo-small/vocab.json",
            "./models/EsperBERTo-small/merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []

        src_files = Path(".data/eval").glob(".txt") if evaluate else Path(".data/train").glob("*.txt")
        for src_file in src_files:
            print("🔥", src_file)
        lines = src_file.read_text(encoding="utf-8").splitlines()
        self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

In [17]:
!python3 run_language_modeling.py --output_dir ./models/EsperBERTo-small-v1\
    --model_type roberta\
    --mlm\
    --config_name ./models/EsperBERTo-small\
    --tokenizer_name ./models/EsperBERTo-small\
    --do_train\
    --do_eval\
    --learning_rate 1e-4\
    --num_train_epochs 5\
    --save_total_limit 2\
    --save_steps 2000\
    --per_gpu_train_batch_size 16\
    --evaluate_during_training\
    --seed 42

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Too bad the blog post did not explain how EsperantoDataset is passed into run_language_modeling.py